In [21]:
import templater3
from wikimapper import WikiMapper
from titlecase import titlecase
import re
import time
import random

In [2]:
test_input = ['<b> spam and eggs </b>', '<b> ham and spam </b>', '<b> white and black </b>']
test_template = Templater()

for document in test_input:
    test_template.learn(document)

print('Template created:', test_template._template)

Template created: [None, '<b> ', None, ' and ', None, ' </b>', None]


In [3]:
data = open('simplewiki-20211120-lists-1k.tsv').readlines()

In [4]:
mapper = WikiMapper("data/index_enwiki-latest.db")

def preprocess_entity(entity: str) -> str:          # thank you antoni
   if entity.find("|"):
      entity = entity.split("|")
      entity = entity[0]

   title_case_entity = titlecase(entity).replace(' ', '_')

   joined = entity.replace(' ', '_')

   capitalized_entity = entity.replace(' ', '_').capitalize()
   
   return title_case_entity, capitalized_entity, joined

def wikiMapper(line):                               # thank you antoni
    entities_counter, counter = 0, 0
    subj, li = line.split('\t')
    items = li.split('\\n')
    for item in items:
        entities = re.findall(r'\[\[(.+?)\]\]',item)
    for entity in entities:
            entities_counter+=1
            title_case, capitalized, joined = preprocess_entity(entity)
            print(title_case, capitalized, joined)
            id = mapper.title_to_id(title_case)
            if id is None:
                id = mapper.title_to_id(capitalized)
                if id is None:
                    id = mapper.title_to_id(joined)
                    if id is None:
                        counter+=1
                        print(title_case, capitalized, joined)
    print(counter/entities_counter)

wikiMapper(data[0])

April_30 April_30 April_30
Walpurgis_Night Walpurgis_night Walpurgis_Night
Europe Europe Europe
0.0


In [5]:
# Search for the item by name
wikidata_id = mapper.title_to_id("Horse_racing")
check = mapper.wikipedia_id_to_title(3342)

# Print the search results
#for result in search_results:
#    print(result['title'], result['id'])
print(wikidata_id)
print(check)

Q187916
Bundesrepublik_Deutschland


In [6]:
from SPARQLWrapper import SPARQLWrapper, JSON

items = "wd:Q43274 wd:Q154952 wd:Q116 wd:Q12097"
print(mapper.id_to_titles('Q43274')[0])
print(mapper.id_to_titles('Q154952')[0])
print(mapper.id_to_titles('Q116')[0])
print(mapper.id_to_titles('Q12097')[2])

Charles_III
Willem-Alexander_of_the_Netherlands
Monarch
King


In [7]:
def query_relations(items: str):
    # SPARQL endpoint URL
    endpoint_url = "https://query.wikidata.org/sparql"

    # SPARQL query, we are querying all 3 items as subject OR object in relation to each other. looking for predicate
    query = '''
    SELECT ?s ?sLabel ?p ?pLabel ?o ?oLabel WHERE {
      VALUES ?s { '''+ items + ''' } VALUES ?o { ''' + items + ''' }
      ?s ?_p ?o .
      ?p wikibase:directClaim ?_p .
      FILTER(?s != ?o)
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    '''

    # Create a SPARQLWrapper instance and set the endpoint URL
    sparql = SPARQLWrapper(endpoint_url)

    # Set the SPARQL query
    sparql.setQuery(query)

    # Set the return format to JSON
    sparql.setReturnFormat(JSON)

    # Execute the SPARQL query
    results = sparql.query().convert()

    # Process the query results
    for result in results['results']['bindings']:
        subject = result['s']['value']
        subject_label = result['sLabel']['value']
        predicate = result['p']['value']
        predicate_label = result['pLabel']['value']
        obj = result['o']['value']
        obj_label = result['oLabel']['value']

        # Print the formatted result
        print("Subject:")
        print("  ID:", subject)
        print("  Label:", subject_label)
        print("Predicate:")
        print("  ID:", predicate)
        print("  Label:", predicate_label)
        print("Object:")
        print("  ID:", obj)
        print("  Label:", obj_label)
        print()

query_relations(items)
    

Subject:
  ID: http://www.wikidata.org/entity/Q154952
  Label: Willem-Alexander of the Netherlands
Predicate:
  ID: http://www.wikidata.org/entity/P106
  Label: occupation
Object:
  ID: http://www.wikidata.org/entity/Q116
  Label: monarch

Subject:
  ID: http://www.wikidata.org/entity/Q12097
  Label: king
Predicate:
  ID: http://www.wikidata.org/entity/P279
  Label: subclass of
Object:
  ID: http://www.wikidata.org/entity/Q116
  Label: monarch



In [8]:
line = data[0]
page, code = line.split('\t')
code = code.replace('\\n','\n')
code = code.split('\n')

In [9]:
def extract_double_square_brackets(text):
    pattern = r'\[\[(.*?)\]\]'
    matches = re.findall(pattern, text)
    return matches

In [10]:
def wikiMapper(entities):                               # thank you antoni
    save = []
    for entity in entities:
        if ':' in entity:
            entity = entity.split(":")
            entity = entity[1]
        title_case, capitalized, joined = preprocess_entity(entity)
        id = mapper.title_to_id(title_case)
        if id is None:
            id = mapper.title_to_id(capitalized)
            if id is None:
                id = mapper.title_to_id(joined)
                if id is None:
                    print('ERROR: ', entity)
        save.append(id)
    return save


for item in code:
    links = extract_double_square_brackets(item)
    save = wikiMapper(links)
    q = ''
    for i in save:
        if i is not None:
            q = 'wd:' + i + ' ' + q
    if len(save) > 1:
        print(q)
        #Waits 1 second
        try:
            query_relations(q)
        except:
            print('Not Found')
    print('--------------------------------')
    

wd:Q80949 wd:Q2510 
Subject:
  ID: http://www.wikidata.org/entity/Q80949
  Label: April Fools' Day
Predicate:
  ID: http://www.wikidata.org/entity/P837
  Label: day in year for periodic occurrence
Object:
  ID: http://www.wikidata.org/entity/Q2510
  Label: April 1

--------------------------------
wd:Q794 wd:Q2510 
--------------------------------
--------------------------------
wd:Q342041 wd:Q869 wd:Q2511 
--------------------------------
wd:Q1436063 wd:Q2511 
--------------------------------
wd:Q414 wd:Q2511 
--------------------------------
wd:Q1041 wd:Q2508 
--------------------------------
--------------------------------
wd:Q916 wd:Q2508 
--------------------------------
wd:Q145 wd:Q8161 wd:Q2507 
--------------------------------
wd:Q30 wd:Q16 wd:Q2506 
Subject:
  ID: http://www.wikidata.org/entity/Q30
  Label: United States of America
Predicate:
  ID: http://www.wikidata.org/entity/P47
  Label: shares border with
Object:
  ID: http://www.wikidata.org/entity/Q16
  Label: Canada


In [11]:
query_relations('wd:Q80949 wd:Q2510')

Subject:
  ID: http://www.wikidata.org/entity/Q80949
  Label: April Fools' Day
Predicate:
  ID: http://www.wikidata.org/entity/P837
  Label: day in year for periodic occurrence
Object:
  ID: http://www.wikidata.org/entity/Q2510
  Label: April 1



In [13]:
def query_type(ids: str):
    big = []
    
    for ID in ids:
        if ID is None:
            continue
        small = []
        # SPARQL endpoint URL
        endpoint_url = "https://query.wikidata.org/sparql"

        # SPARQL query to retrieve the type of the given item
        query = '''
        SELECT ?item ?itemLabel ?itemDescription ?type ?typeLabel
        WHERE {
          VALUES ?item { wd:''' + ID + ''' }
          ?item wdt:P31 ?type .
          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
        }
        '''

        # Create a SPARQLWrapper instance and set the endpoint URL
        sparql = SPARQLWrapper(endpoint_url)

        # Set the SPARQL query
        sparql.setQuery(query)

        # Set the return format to JSON
        sparql.setReturnFormat(JSON)

        # Execute the SPARQL query
        results = sparql.query().convert()

        # Process the query results
        for result in results['results']['bindings']:
            item = result['item']['value']
            item_label = result['itemLabel']['value']
            item_description = result['itemDescription']['value']
            item_type = result['type']['value']
            item_type_label = result['typeLabel']['value']
            
            
            small.append(item_type_label)
            
        big.append(small)
    final = []
    #print(big)
    for s in big:
        if s != []:
            final.append(min(s, key=len))
    return final
    
    
def replaceType(text: str):
    pattern = r'\[\[(.*?)\]\]'
    matches = re.findall(pattern, text)
    things = wikiMapper(matches)
    new = query_type(things)
    for index, m in enumerate(matches):
        if len(new) > index:
            text = text.replace(m, new[index])
    return text


thingyy = []
for item in code:
    print(item)
    save = replaceType(item)
    if save != '':
        thingyy.append(save)
    #print(save)
print(thingyy)

* [[April 1]] - [[April Fools' Day]]
* [[April 1]] - Islamic Republic Day ([[Iran]])
* [[April 2]] - International Children's Book Day
* [[April 2]] - [[Thailand|Thai]] Heritage and [[wikt:conservation|Conservation]] Day
* [[April 2]] - World [[Autism]] Awareness Day
* [[April 2]] - Malvinas Day ([[Argentina]])
* [[April 4]] - Independence Day ([[Senegal]])
* [[April 4]] - International Day for Landmine Awareness and Assistance
* [[April 4]] - Peace Day ([[Angola]])
* [[April 5]] - End of [[Tax]] Year ([[United Kingdom]])
* [[April 6]] - Tartan Day ([[Canada]] and [[United States]])
* [[April 6]] - Chakri Day ([[Thailand]])
* [[April 7]] - Day of Maternity and Beauty ([[Armenia]])
* [[April 7]] - Genocide Memorial Day ([[Rwanda]])
* [[April 7]] - World [[Health]] Day
* [[April 7]] - Women's Day ([[Mozambique]])
* [[April 8]] - [[Buddha]]'s Birthday ([[Buddhism]])
* [[April 9]] - Martyrs' Day ([[Tunisia]])
* [[April 9]] - Day of National Unity ([[Georgia (country)|Georgia]])
* [[April 9

query page + link? 

In [14]:
t3st = Templater(min_block_size=1)

for row in thingyy:
    print(row)
    t3st.learn(row)

print('Template created:', t3st._template)

* [[point in time with respect to recurrent timeframe]] - [[recurring event]]
* [[point in time with respect to recurrent timeframe]] - Islamic Republic Day ([[country]])
* [[point in time with respect to recurrent timeframe]] - International Children's Book Day
* [[point in time with respect to recurrent timeframe]] - [[country]] Heritage and [[Wikimedia disambiguation page]] Day
* [[point in time with respect to recurrent timeframe]] - World [[disability]] Awareness Day
* [[point in time with respect to recurrent timeframe]] - Malvinas Day ([[nation]])
* [[point in time with respect to recurrent timeframe]] - Independence Day ([[country]])
* [[point in time with respect to recurrent timeframe]] - International Day for Landmine Awareness and Assistance
* [[point in time with respect to recurrent timeframe]] - Peace Day ([[country]])
* [[point in time with respect to recurrent timeframe]] - End of [[realm]] Year ([[United Kingdom]])
* [[point in time with respect to recurrent timeframe

In [15]:
lim = 0
for row in data:
    if lim > 10:
        break
    lim += 1
    p, c = row.split('\t')
    print(p)
    print(c)

April
* [[April 1]] - [[April Fools' Day]]\n* [[April 1]] - Islamic Republic Day ([[Iran]])\n* [[April 2]] - International Children's Book Day\n* [[April 2]] - [[Thailand|Thai]] Heritage and [[wikt:conservation|Conservation]] Day\n* [[April 2]] - World [[Autism]] Awareness Day\n* [[April 2]] - Malvinas Day ([[Argentina]])\n* [[April 4]] - Independence Day ([[Senegal]])\n* [[April 4]] - International Day for Landmine Awareness and Assistance\n* [[April 4]] - Peace Day ([[Angola]])\n* [[April 5]] - End of [[Tax]] Year ([[United Kingdom]])\n* [[April 6]] - Tartan Day ([[Canada]] and [[United States]])\n* [[April 6]] - Chakri Day ([[Thailand]])\n* [[April 7]] - Day of Maternity and Beauty ([[Armenia]])\n* [[April 7]] - Genocide Memorial Day ([[Rwanda]])\n* [[April 7]] - World [[Health]] Day\n* [[April 7]] - Women's Day ([[Mozambique]])\n* [[April 8]] - [[Buddha]]'s Birthday ([[Buddhism]])\n* [[April 9]] - Martyrs' Day ([[Tunisia]])\n* [[April 9]] - Day of National Unity ([[Georgia (country

In [18]:
def generate_numbers(size):
    numbers = random.sample(range(1001), size)
    return numbers

chosen = generate_numbers(40)

In [19]:
print(chosen)

[568, 432, 923, 955, 611, 258, 431, 20, 215, 997, 461, 466, 819, 366, 167, 470, 673, 200, 59, 294, 158, 545, 595, 651, 398, 716, 731, 182, 172, 333, 170, 91, 527, 266, 70, 60, 119, 429, 389, 747]


In [23]:
for number in chosen:
    line = open('simplewiki-20211120-lists-1k.tsv').readlines()[number]
    page, code = line.split('\t')
    code = code.replace('\\n', '\n').splitlines()
    print(code)
    save_array = []
    for item in code:
        save = replaceType(item)
        if save != '':
            save_array.append(save)
    print(save_array)
    templater = templater3.Templater(min_block_size=2)
    for individual in save_array:
        templater.learn(individual)
    print(templater._template)
    print('----------------------')

['* [[Addition]]', '* [[Division (mathematics)]]', '* [[Multiplication]]', '* [[Negative number]]', '* [[Order of operations]]']
['* [[hyperoperation]]', '* [[Division (mathematics)]]', '* [[hyperoperation]]', '* [[type of number]]', '* [[Order of operations]]']
[None, '* [[', None, ']]', None]
----------------------
['* [[November 1]], [[1512]] {{ndash}} The ceiling of the [[Sistine Chapel]], painted by [[Michelangelo]], goes on display.', '* [[November 1]], [[1755]] {{ndash}} An [[earthquake]] and [[tsunami]]s kills thousands of people in [[Lisbon]], and affect other parts of [[Europe]] and North [[Africa]].', '* [[November 1]], [[1922]] {{ndash}} [[Mehmed VI]], the last [[Sultan]] of the [[Ottoman Empire]], leaves the throne.', '* [[November 1]], [[1981]] {{ndash}} [[Antigua and Barbuda]] become independent from the [[United Kingdom]].', '* [[November 1]], [[1993]] {{ndash}} The [[European Union]] is officially created.', '* [[November 2]], [[1889]] {{ndash}} [[North Dakota|North]] 